In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%reload_ext autoreload
%autoreload 1

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

import os, glob
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, euclidean
from scipy import stats
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

import statsmodels.formula.api as smfapi
import statsmodels.api as smapi
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as MSE

from scipy.stats import f as f_test

from Utils.UtilsScraping import *

In [95]:
it

3

In [96]:
import time
import re, string


game_id = 292030

page_url_header = "https://steamcommunity.com/app/" + str(game_id) + "/homecontent/?"


page_url_params = {
    "userreviewsoffset":10,
    "p":1,
    "workshopitemspage": 1,
     "readytouseitemspage": 1,
     "mtxitemspage": 1,
     "itemspage": 1,
     "screenshotspage": 1,
     "videospage": 1,
     "artpage": 1,
     "allguidepage": 1,
     "webguidepage": 1,
     "integratedguidepage": 1,
     "discussionspage": 1,
}

page_url_params_frozen = {
    "numperpage": 10,
    "browsefilter": "mostrecent",
    "appid": "292030",
    "l": "english",
    "appHubSubSection": 10,
    "filterLanguage": "default",
    "searchText": "",
    "forceanon": 1
}

# Make the first page URL
page_urls = ["https://steamcommunity.com/app/" + str(game_id) + "/reviews/?browsefilter=mostrecent&snr=1_5_100010_&p=1"]
if it > 0:
    page_urls = []
# Generate the remaining pages
for page_no in range(int(0+it*100), int(100*(it+1))):
    my_url = page_url_header
    for k, v in page_url_params.items():

        if ("userreviewsoffset" == k) and (page_no > 0):
            my_url += k
            my_url += "="
            my_url += str(v * page_no)
            my_url += "&"
            continue

        my_url += k
        my_url += "="
        my_url += str(v + page_no)
        my_url += "&"

    for k, v in page_url_params_frozen.items():
        my_url += k
        my_url += "="
        my_url += str(v)
        my_url += "&"

    my_url = my_url[:-1]

    page_urls.append(my_url)


all_reviews = []

for my_url in page_urls:
    print(my_url)
    
    page_soup = get_page_soup(my_url)

    div_cards = page_soup.findAll(name="div", attrs={"class":"apphub_CardContentMain"})
    
    print("Number of cards : ", len(div_cards))

    for card in div_cards:
        found_helpful = card.findAll(name="div", attrs={"class":"found_helpful"})[0].text
        found_helpful = found_helpful.strip()
        flag_recommended = card.findAll(name="div", attrs={"class":"title"})[0].text
        playtime = card.findAll(name="div", attrs={"class":"hours"})[0].text
        date_posted = card.findAll(name="div", attrs={"class":"date_posted"})[0].text
        unclean_review = card.text
        review = ""
        for ucr in unclean_review.split("\t"):
            if (date_posted == ucr) or (len(ucr)==0) or (found_helpful == ucr):
                continue
            review += ucr
            review += " "
        # Sanity check - remove trailing white space and other text elements
        review = review.split(date_posted)[1]
        review = review.strip()
        all_reviews.append((date_posted, playtime, review, found_helpful, flag_recommended))

    time.sleep(1)
#     break

https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3000&p=301&workshopitemspage=301&readytouseitemspage=301&mtxitemspage=301&itemspage=301&screenshotspage=301&videospage=301&artpage=301&allguidepage=301&webguidepage=301&integratedguidepage=301&discussionspage=301&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3010&p=302&workshopitemspage=302&readytouseitemspage=302&mtxitemspage=302&itemspage=302&screenshotspage=302&videospage=302&artpage=302&allguidepage=302&webguidepage=302&integratedguidepage=302&discussionspage=302&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3020&p=303&workshopitemspage=303&readytouseitemspage=303&mtxitemspa

https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3190&p=320&workshopitemspage=320&readytouseitemspage=320&mtxitemspage=320&itemspage=320&screenshotspage=320&videospage=320&artpage=320&allguidepage=320&webguidepage=320&integratedguidepage=320&discussionspage=320&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3200&p=321&workshopitemspage=321&readytouseitemspage=321&mtxitemspage=321&itemspage=321&screenshotspage=321&videospage=321&artpage=321&allguidepage=321&webguidepage=321&integratedguidepage=321&discussionspage=321&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3210&p=322&workshopitemspage=322&readytouseitemspage=322&mtxitemspa

https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3380&p=339&workshopitemspage=339&readytouseitemspage=339&mtxitemspage=339&itemspage=339&screenshotspage=339&videospage=339&artpage=339&allguidepage=339&webguidepage=339&integratedguidepage=339&discussionspage=339&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3390&p=340&workshopitemspage=340&readytouseitemspage=340&mtxitemspage=340&itemspage=340&screenshotspage=340&videospage=340&artpage=340&allguidepage=340&webguidepage=340&integratedguidepage=340&discussionspage=340&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3400&p=341&workshopitemspage=341&readytouseitemspage=341&mtxitemspa

https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3570&p=358&workshopitemspage=358&readytouseitemspage=358&mtxitemspage=358&itemspage=358&screenshotspage=358&videospage=358&artpage=358&allguidepage=358&webguidepage=358&integratedguidepage=358&discussionspage=358&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3580&p=359&workshopitemspage=359&readytouseitemspage=359&mtxitemspage=359&itemspage=359&screenshotspage=359&videospage=359&artpage=359&allguidepage=359&webguidepage=359&integratedguidepage=359&discussionspage=359&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3590&p=360&workshopitemspage=360&readytouseitemspage=360&mtxitemspa

https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3760&p=377&workshopitemspage=377&readytouseitemspage=377&mtxitemspage=377&itemspage=377&screenshotspage=377&videospage=377&artpage=377&allguidepage=377&webguidepage=377&integratedguidepage=377&discussionspage=377&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3770&p=378&workshopitemspage=378&readytouseitemspage=378&mtxitemspage=378&itemspage=378&screenshotspage=378&videospage=378&artpage=378&allguidepage=378&webguidepage=378&integratedguidepage=378&discussionspage=378&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3780&p=379&workshopitemspage=379&readytouseitemspage=379&mtxitemspa

https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3950&p=396&workshopitemspage=396&readytouseitemspage=396&mtxitemspage=396&itemspage=396&screenshotspage=396&videospage=396&artpage=396&allguidepage=396&webguidepage=396&integratedguidepage=396&discussionspage=396&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3960&p=397&workshopitemspage=397&readytouseitemspage=397&mtxitemspage=397&itemspage=397&screenshotspage=397&videospage=397&artpage=397&allguidepage=397&webguidepage=397&integratedguidepage=397&discussionspage=397&numperpage=10&browsefilter=mostrecent&appid=292030&l=english&appHubSubSection=10&filterLanguage=default&searchText=&forceanon=1
Number of cards :  10
https://steamcommunity.com/app/292030/homecontent/?userreviewsoffset=3970&p=398&workshopitemspage=398&readytouseitemspage=398&mtxitemspa

In [97]:
reviews_df = pd.DataFrame(all_reviews)
reviews_df.columns = ["date_posted", "playtime", "review", "helpful_funny", "recommended"]

reviews_df.to_csv("./reviews_witcher3_" + str(it) + ".csv")
it += 1

In [98]:
rdf0 = pd.read_csv("./reviews_witcher3_0.csv", index_col="Unnamed: 0")
rdf1 = pd.read_csv("./reviews_witcher3_1.csv", index_col="Unnamed: 0")
rdf2 = pd.read_csv("./reviews_witcher3_3.csv", index_col="Unnamed: 0")
# rdf = pd.read_csv("./reviews_gears5.csv", index_col="Unnamed: 0")
display(rdf0.shape, rdf1.shape, rdf2.shape)

(1010, 5)

(1000, 5)

(1000, 5)

In [99]:
it

4

-----------

-----------

-----------